# KNN. Выбор числа соседей

###### Введение
Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

- Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
- Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
- При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному

Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

Метод k ближайших соседей реализован в классе sklearn.neighbors.KNeighborsClassifier. Основным параметром является n_neighbors, который задает число соседей для построения прогноза.

Кросс-валидация заключается в разделении выборки на m непересекающихся блоков примерно одинакового размера, после чего выполняется m шагов. На i-м шаге i-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки. Соответственно, на каждом шаге алгоритм обучается на некоторой обучающей выборке, после чего вычисляется его качество на тестовой выборке. После выполнения m шагов мы получаем m показателей качества, усреднение которых и дает оценку кросс-валидации. 

Технически кросс-валидация проводится в два этапа:
- Создается генератор разбиений sklearn.model_selection.KFold, который задает набор разбиений на обучение и валидацию. Число блоков в кросс-валидации определяется параметром n_splits. Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору KFold параметр shuffle=True.
- Вычислить качество на всех разбиениях можно при помощи функции sklearn.model_selection.cross_val_score. В качестве параметра estimator передается классификатор, в качестве параметра cv — генератор разбиений с предыдущего шага. С помощью параметра scoring можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (accuracy). Результатом является массив, значения которого нужно усреднить.

Приведение признаков к одному масштабу можно делать с помощью функции sklearn.preprocessing.scale, которой на вход необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение.

###### В этом задании вам нужно подобрать оптимальное значение k для алгоритма kNN. Будем использовать набор данных Wine, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.

In [5]:
import pandas as pd
import numpy as np

In [36]:
#data = np.genfromtxt('wine.data', delimiter=',')
data = pd.read_csv('wine.data', index_col=False, header=None)
data.columns = ['type',
                'alcohol',
                'malic_acid',
                'ash',
                'alcalinity_of_ash',
                'magnesium',
                'total_phenols',
                'flavanoids',
                'nonflavanoid_phenols',
                'proanthocyanins',
                'color_intensity',
                'hue',
                'of_diluted_wines',
                'proline']

In [37]:
y = data['type']
X = data.iloc[:,1:]

Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

In [21]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [38]:
cv = KFold(n_splits=5, shuffle=True, random_state=42) # разбиение выборки

Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.

In [51]:
max_q = -1 # тут храним максимальное качество модели
opt_k = 0 # тут храним k соседей при этом качестве
for k in range(1,51):    
    clf = KNeighborsClassifier(n_neighbors=k) # KNN с разными k
    q = np.mean(cross_val_score(clf, X, y, cv=cv, scoring='accuracy')) # выдает 5 значений, которые усредняем
    if q > max_q:
        max_q = q
        opt_k = k

In [57]:
print(f'opt k: {opt_k}')
print(f'max quality: {max_q}')

opt k: 1
max quality: 0.7304761904761905


Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кросс-валидации.

In [58]:
from sklearn.preprocessing import scale

In [59]:
X_st = scale(X)

In [60]:
max_q = -1 # тут храним максимальное качество модели
opt_k = 0 # тут храним k соседей при этом качестве
for k in range(1,51):    
    clf = KNeighborsClassifier(n_neighbors=k) # KNN с разными k
    q = np.mean(cross_val_score(clf, X_st, y, cv=cv, scoring='accuracy')) # выдает 5 значений, которые усредняем
    if q > max_q:
        max_q = q
        opt_k = k

In [61]:
print(f'opt k: {opt_k}')
print(f'max quality: {max_q}')

opt k: 29
max quality: 0.9776190476190475
